In [1]:

from torch import nn
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoTokenizer
from torch.optim import Adam
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import requests
from bs4 import BeautifulSoup
import os

d:\Program Files (x86)\Miniconda3\envs\Masterarbeit\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 36.5 MB/s 
     |████████████████████████████████| 120 kB 68.1 MB/s 


In [ ]:
torch.cuda.empty_cache()
pd.set_option('display.max_rows',500)




'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3549 MB |    3573 MB |    5711 MB |    2162 MB |\n|       from large pool |    3548 MB |    3572 MB |    5710 MB |    2162 MB |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3549 MB |    3573 MB |    5711 MB |    2162 MB |\n|       from large pool |    3548 MB |    3572 MB |

In [ ]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [ ]:
### Merging Sentiment and Opinion

dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic["Sentiment"] = dataset_topic["Sentiment"] + dataset_topic["Opinion"]
label_to_id = {x:y for y,x in enumerate(dataset_topic["Sentiment"].unique()) }
dataset_topic = dataset_topic[["Comment", "Sentiment"]]
df_train, df_test = np.split(dataset_topic.sample(frac=1, random_state=77), [int(.8*len(dataset_topic))])
dataset_topic.reset_index(drop=True, inplace=True)
dataset_topic["Sentiment"].unique()
df_train


,Comment,Sentiment
401,Die Einigung für Schulen ist fatal. Frau Eisen...,negative
478,"Vernünftige Entscheidung, mit der Impfung entf...",positive
2488,Wir sind doch nicht in einer (Standard-) Schul...,negative
843,"Wie schön, es geht voran. Bis Ende Mai sind ...",positive
344,"Nun gut, wenn sarkastische Kommentare nicht me...",neutral
...,...,...
1066,Übrigens: das RKI prognostizierte am 12. März ...,neutral
3901,Donny will nach 8 Monaten beginnen Menschen zu...,negative
1393,De lockdau mi sine positive eigeschaft für d'u...,negative
3027,"Deutschlands ""Intelligentia"" regelt ...",negative


In [24]:
augmented_400  =pd.read_csv("annotated_data/data_augment_400_positive.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Comment"], encoding="utf-8", header = None)
augmented_400  = augmented_400.sample(frac=1, random_state=77).reset_index(drop=True)
augmented_400.insert(4, 'Sentiment', ["positive" for i in range(len(augmented_400))])
augmented_400 = augmented_400[["Comment", "Sentiment"]]



# dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_final_topics.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
# label_to_id = {x:y for y,x in enumerate(dataset_topic["Sentiment"].unique()) }
# dataset_topic = dataset_topic.loc[dataset_topic["Sentiment"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic[["Comment", "Sentiment"]]
# df_train, df_test = np.split(dataset_topic.sample(frac=1, random_state=77), [int(.8*len(dataset_topic))])


# df_train = pd.concat([df_train, augmented_400])
# df_train = df_train.sample(frac=1, random_state=77).reset_index(drop=True)
# df_train


In [4]:
### Here we set the labels of the topics, in this case we have 7 labels for comments and 8 for articles
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_final_topics.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick": 7}
label_to_id = {x:y for y,x in enumerate(dataset_topic["Sentiment"].unique()) }
#label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6}
# label_to_id = {"positive":0, "negative" : 1, "neutral": 2}
print(label_to_id)
# dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleanedArticleText.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="utf-8")
# dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_final_topics.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")


print(dataset_topic["Topic_comment"].value_counts()[dataset_topic["Topic_comment"].value_counts() < 200].index.to_list())
print(dataset_topic["Topic_comment"].value_counts())
dataset_topic = dataset_topic.loc[dataset_topic["Opinion"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_comment"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
print(dataset_topic)
# dataset_topic = dataset_topic[["Comment", "Topic_comment"]]
dataset_topic = dataset_topic[["Comment", "Sentiment"]]
# dataset_topic = dataset_topic[["Comment", "Topic_article"]]

# dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
# dataset_topic = dataset_topic.loc[dataset_topic["Topic_comment"].isin(label_to_id.keys())]
dataset_topic = dataset_topic.loc[dataset_topic["Sentiment"].isin(label_to_id.keys())]

# dataset_topic.columns = ["Comment", "Topic"]
dataset_topic.columns = ["Comment", "Sentiment"]
print(len(dataset_topic))
df_train, df_test = np.split(dataset_topic.sample(frac=1, random_state=77), [int(.8*len(dataset_topic))])


##### Concats 400 positive samples
# df_train = pd.concat([df_train, augmented_400])
# df_train = df_train.sample(frac=1, random_state=77).reset_index(drop=True)

len(dataset_topic)
dataset_topic.reset_index(drop=True, inplace=True)

df_train.drop(df_train.loc[df_train["Comment"].isnull()].index, axis = 0, inplace = True)
# for x in df_train["Comment"]:
#     if type(x) != str:
#         print(type(x), x)
df_train


{'negative': 0, 'positive': 1, 'neutral': 2}
['demos']
maßnahmen     1110
politik        915
infektion      571
impfung        549
wirtschaft     449
lockerung      288
demos          114
Name: Topic_comment, dtype: int64
                                        ID        Date      Time  \
0     40a7c181-1ab8-4d4b-bca5-bf8f4ace27ea  30.04.2020  06:59:00   
1     4736f7ec-6396-4f5f-991b-1f83e5f5df66  07.03.2021  17:19:00   
2     5994b2b8-4e42-49a7-b82c-c78e084df7f7  18.03.2021  13:40:00   
3     15557a4c-0e90-412e-8bd5-03ae3f6a7854  13.02.2021  17:57:00   
4     aecb14c3-12f5-4f76-aab8-c3620a709fa4  22.12.2020  10:18:00   
...                                    ...         ...       ...   
3991  855c1ae5-aa3b-4497-bf62-a2b450564320  18.03.2021  16:23:00   
3992  3d7819e6-f105-4d41-a2c4-1d06b6f51dc1  22.05.2020  07:15:00   
3993  2fbfe25d-a788-409a-844d-fb7bd80bcf44  09.01.2021  13:26:00   
3994  55863de1-bf74-49d1-95fe-08d2fbd51970  15.04.2021  11:50:00   
3995  bd0465c0-4fc7-4762-9731-

,Comment,Sentiment
401,Die Einigung für Schulen ist fatal. Frau Eisen...,negative
478,"Vernünftige Entscheidung, mit der Impfung entf...",positive
2488,Wir sind doch nicht in einer (Standard-) Schul...,negative
843,"Wie schön, es geht voran. Bis Ende Mai sind ...",positive
344,"Nun gut, wenn sarkastische Kommentare nicht me...",neutral
...,...,...
1066,Übrigens: das RKI prognostizierte am 12. März ...,neutral
3901,Donny will nach 8 Monaten beginnen Menschen zu...,negative
1393,De lockdau mi sine positive eigeschaft für d'u...,negative
3027,"Deutschlands ""Intelligentia"" regelt ...",negative


In [2]:
# This cell scrapes the articles for their header content

label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "wirtschaft":4, "lockerung":5, "demos":6, "überblick":7}
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
dataset_topic.reset_index(drop=True, inplace=True)
for x in range(len(dataset_topic)):
    print(x)

    request = requests.get("https://www.spiegel.de/wissenschaft/medizin/corona-news-am-samstag-die-wichtigsten-entwicklungen-zu-sars-cov-2-und-covid-19-a-" +dataset_topic.loc[int(x), "ID"])
    soup = BeautifulSoup(request.content, "html.parser")
    title = soup.find("title").text
    subtitle = soup.find("meta", property="og:description")["content"]

    dataset_topic.loc[x, "Comment"] = title + " " + subtitle

dataset_topic


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,ID,Date,Time,Comment Level,Username,Opinion,Sentiment,Topic_comment,Topic_article,Comment,Method
0,40a7c181-1ab8-4d4b-bca5-bf8f4ace27ea,2020-04-30,06:59:00,0,Karsten-QGRAZB3WR,negative,negative,lockerung,lockerung,Corona-Lockdown-Lockerungen: Deutsche suchen w...,manual
1,4736f7ec-6396-4f5f-991b-1f83e5f5df66,2021-03-07,17:19:00,0,Hospitalitaet,positive,positive,infektion,überblick,Corona-News am 07.03.: Olaf Scholz verspricht ...,manual_al
2,5994b2b8-4e42-49a7-b82c-c78e084df7f7,2021-03-18,13:40:00,0,Karl-Heinz-lFYMd87Mg,negative,negative,infektion,überblick,Corona-News am 18.3.: Länder setzen Öffnungssc...,manual_al
3,aa90b76d-f535-43bc-81da-759ff0a1f610,06.04.2021,18:01:00,0,Thomas-opcCIf_Gg,negative,negative,lockerung,politik,Rezo kritisiert die Corona-Politik: »So ein Ha...,manual_al
4,a24ee383-2310-4166-ad8b-4250259ada77,2021-04-06,08:55:00,0,Georg-e2NHZx9Zg,negative,negative,wirtschaft,wirtschaft,EU-Kommission genehmigt Finanzhilfen für Air F...,manual_al
...,...,...,...,...,...,...,...,...,...,...,...
2983,899443b1-43a6-4048-b7fc-f2d3b905ca87,2020-11-27,06:32:00,0,Tullius,negative,negative,krankenhaus,überblick,"Corona-News am 27.11.: Frankreich, Italien, Ir...",manual_al
2984,ebc7c5fe-11de-4b3e-b7cd-3a9f50fc51e2,2020-09-29,12:23:00,0,JamesRealistano,neutral,neutral,wirtschaft,wirtschaft,Corona-Krise: Mehrheit der Deutschen unterstüt...,manual_al
2985,0210360b-673b-46a4-bb95-4998e452f1a4,2020-04-01,05:31:00,0,Jan-6-zu_9rWg,negative,negative,wirtschaft,wirtschaft,Coronavirus: SPD-Chefin Saskia Esken fordert S...,manual_al
2986,3c9cef10-a61a-429a-a5e4-271b2d2f34bd,2020-08-04,02:38:00,0,Christina-XMNOcI7GR,positive,neutral,demos,überblick,Coronavirus News am 4.8.: Novavax berichtet vo...,manual


In [36]:

dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_ArticleHeader.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic

,ID,Date,Time,Comment Level,Username,Opinion,Sentiment,Topic_comment,Topic_article,Comment,Method
0,40a7c181-1ab8-4d4b-bca5-bf8f4ace27ea,2020-04-30,06:59:00,0,Karsten-QGRAZB3WR,negative,negative,lockerung,lockerung,SPON macht mit bei der Hetze der Covid19 Leugn...,manual
1,4736f7ec-6396-4f5f-991b-1f83e5f5df66,2021-03-07,17:19:00,0,Hospitalitaet,positive,positive,infektion,überblick,+++ Britische Forscher infizieren Freiwillige ...,manual_al
2,5994b2b8-4e42-49a7-b82c-c78e084df7f7,2021-03-18,13:40:00,0,Karl-Heinz-lFYMd87Mg,negative,negative,infektion,überblick,"Braucht es noch einen weiteren Nachweis, dass ...",manual_al
3,15557a4c-0e90-412e-8bd5-03ae3f6a7854,13.02.2021,17:57:00,0,Glaubnichtalles,neutral,neutral,virus,virus,Es sollte die Frage gestellt werden. Was ist d...,manual_al
4,aecb14c3-12f5-4f76-aab8-c3620a709fa4,2020-12-22,10:18:00,0,Schmandter-JT6IwOpMR,negative,negative,virus,virus,Es wird bis in den Januar hinein Untersuchunge...,manual_al
...,...,...,...,...,...,...,...,...,...,...,...
3991,855c1ae5-aa3b-4497-bf62-a2b450564320,2021-03-18,16:23:00,0,M.-wz918EnMg,positive,neutral,impfung,impfung,Ich halte die Freigabe trotz allem für vertret...,manual
3992,3d7819e6-f105-4d41-a2c4-1d06b6f51dc1,2020-05-22,07:15:00,0,Rüdiger-GQduR_6WR,negative,neutral,china,china,"Ich hoffe, dass das bald ein Ende hat. China b...",manual_al
3993,2fbfe25d-a788-409a-844d-fb7bd80bcf44,2021-01-09,13:26:00,0,Coraggioso-Ir6WKaaMR,positive,negative,impfung,lockdown,"Merkel: ""Ein Virus, das uns alle trifft, lässt...",manual
3994,55863de1-bf74-49d1-95fe-08d2fbd51970,15.04.2021,11:50:00,0,Maria-_AnWpnTWg,neutral,negative,sport,sport,"Moin, schon komisch, dass Vereine in den unt...",manual_al


In [ ]:
### Saves the dataframe with comment cells filled with article text. Sadly not all articles contrained the
### tag searched with soup, so there are some missing articles in the training data set
dataset_topic.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_article.csv", encoding="utf-8", index=False, header=False)

In [5]:
data_text = []
data_labels = []
text_train = []
labels_train = []
text_test = np.array([])
labels_test = np.array([])


tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")



# For Topics Article
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick":7 }
# id_to_label = {0:"politik", 1:"maßnahmen", 2:"infektion", 3: "impfung", 4: "lockdown", 5:"wirtschaft", 6:"lockerung", 7:"überblick"}


# For Topics Comment
# label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6}
# id_to_label = {0:"politik", 1:"maßnahmen", 2:"infektion", 3: "impfung", 4: "lockdown", 5:"wirtschaft", 6:"lockerung"}

# For Sentiment
# label_to_id = {"positive":0, "negative" : 1, "neutral": 2}
# id_to_label = {0:"positive", 1:"negative", 2 : "neutral"}


label_to_id = {x:y for y,x in enumerate(dataset_topic["Sentiment"].unique()) }
id_to_label = {x:y for x,y in enumerate(dataset_topic["Sentiment"].unique()) }


### Here we create a training set that can be used to compare across different sizes of training data



class Dataset(torch.utils.data.Dataset):

    def __init__(self,dataframe):
#         self.labels = [label_to_id[label] for label in dataframe["Topic"] if label in ["politik", "maßnahmen", "infektion", "impfung", "lockdown", "wirtschaft", "lockerung", "überblick"]]
        self.labels = [label_to_id[label] for label in dataframe["Sentiment"] if label in label_to_id.keys()]
        self.texts = [tokenizer(txt, padding ="max_length", max_length = 512, truncation=True, return_tensors="pt") for txt in dataframe["Comment"]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_texts, batch_labels
# ### Define test and train set
# text_test = text_train[split_index+1:]
# text_train = text_train[:split_index]
# labels_test = labels_train[split_index+1:]
# labels_train = labels_train[:split_index]


### Replace label as int
for idx, labels in enumerate(data_labels):
    data_labels[idx] = label_to_id[labels]


class ClassifierText(nn.Module):
    def __init__(self, dropout = 0.7):
        super(ClassifierText, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-german-cased")
        self.dropout = nn.Dropout(dropout)
        
        # For Topic Article
#         self.linear = nn.Linear(768, 8)
        # For Topic Comment
        # self.linear = nn.Linear(768, 7)
        # For Sentiment
        self.linear = nn.Linear(768, 3)
#         self.relu = nn.ReLU()
        # For SentimentOpinion
#         self.linear = nn.Linear(768, 20)
        self.relu = nn.ReLU()
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer




Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()


In [6]:
## training loop from repo https://gist.github.com/marcellusruben/e4de016da784f4b2debb6ea6f674867d, modified to fit data set and added tensorboard logs
def train(model, train_data, test_data, learning_rate, epochs):

    train, test = Dataset(train_data), Dataset(test_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") #"cuda:3" if use_cuda else "cpu"
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

                for test_input, test_label in test_dataloader:

                    test_label = test_label.to(device)
                    mask = test_input['attention_mask'].to(device)
                    input_id = test_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, test_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == test_label).sum().item()
                    total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(test_data): .3f} \
            | Val Accuracy: {total_acc_val / len(test_data): .3f}')
        writer.add_scalar("Loss/Train", total_loss_train, epoch_num) 
        writer.add_scalar("Loss/Test", total_loss_val, epoch_num)
        writer.add_scalar("Accuracy/Train", total_acc_train, epoch_num)
        writer.add_scalar("Accuracy/Test", total_acc_val, epoch_num)
            
EPOCHS = 4
model = ClassifierText()
LR = 1e-5

train(model, df_train, df_test, LR, EPOCHS)
# for train_index, test_index in folds.split(data_text, data_labels):
#     text_train, labels_train  = data_text[train_index], data_labels[train_index]
#     text_test, labels_test  = data_text[test_index], data_labels[test_index]

#     train_dict = {"texts": text_train, "labels" : labels_train}
#     test_dict = {"texts": text_test, "labels" : labels_test}              
#     train(model, text_train, LR, EPOCHS)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|          | 31/3196 [03:27<5:52:57,  6.69s/it]


KeyboardInterrupt: ignored

In [43]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
df_test.drop(df_test.loc[df_test["Comment"].isnull()].index, axis = 0, inplace = True)
evaluate(model, df_test)



Test Accuracy:  0.515


In [49]:
torch.save(model.state_dict(),"Model_OpinionFULL.pt")

In [ ]:
model = ClassifierText()
model.load_state_dict(torch.load("Model_SentimentAugmented.pt"))
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)

In [44]:
geladenes_model = model
evaluate(geladenes_model, df_test)

Test Accuracy:  0.505


In [47]:
from numpy.ma.core import argmax
from transformers import TextClassificationPipeline

use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    mein_model = geladenes_model.cuda()

# correct_per_topic = {"politik" : 0, "maßnahmen" : 0, "infektion" : 0, "impfung": 0, "lockdown": 0, "wirtschaft":0, "lockerung":0, "überblick":0}
# correct_per_topic = {"politik" : 0, "maßnahmen" : 0, "infektion" : 0, "impfung": 0, "lockdown": 0, "wirtschaft":0, "lockerung":0}
correct_per_topic = {x:0 for x in label_to_id.keys()}
true = 0
with torch.no_grad():

  for _, sample in df_test.iterrows():
    input_ids = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["input_ids"].to(device)

    masks = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["attention_mask"].to(device)
    
    prediction = model( input_ids, masks)
 
    class_pred = np.argmax(prediction.cpu()).item()
    
    topic = label_to_id[sample["Opinion"]]
#     topic = label_to_id[sample["Sentiment"]]

    #print(class_pred, topic)
    
    
    if class_pred == topic:
      true+= 1
      correct_per_topic[id_to_label[topic]] += 1


print(true/len(df_test))

for topic in df_test["Opinion"].unique():
    correct_per_topic[topic] /= df_test["Opinion"].value_counts()[topic]

    
# for sentiment in df_test["Sentiment"].unique():
#   correct_per_topic[sentiment] /= df_test["Sentiment"].value_counts()[sentiment]

#     prediction = model(mask=masks, input_id=input_ids)

#     topic = np.argmax(prediction)

#     topic
print(correct_per_topic)
# print(df_train["Topic"].value_counts())
print(df_test["Opinion"].value_counts())

0.515
{'negative': 0.7025316455696202, 'positive': 0.0, 'neutral': 0.6148867313915858}
negative    316
neutral     309
positive    175
Name: Opinion, dtype: int64


In [ ]:
print(df_train["Sentiment"].value_counts())

neutral     1543
negative    1511
positive     142
Name: Sentiment, dtype: int64


In [ ]:
pd.set_option("max_rows", None)
meine_daten = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
meine_daten.loc[meine_daten["Topic_article"] == "intensivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "homeoffice", "Topic_article"] = "arbeit"
meine_daten.loc[meine_daten["Topic_article"] == "schule", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "reiseverbot", "Topic_article"] = "reise"
meine_daten.loc[meine_daten["Topic_article"] == "quarantäne", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schulschließung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "demo", "Topic_article"] = "demos"
meine_daten.loc[meine_daten["Topic_article"] == "warnapp", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intensivstationen", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "lockerungen", "Topic_article"] = "lockerung"
meine_daten.loc[meine_daten["Topic_article"] == "konjunktur", "Topic_article"] = "wirtschaft"
meine_daten.loc[meine_daten["Topic_article"] == "schulschliessung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "inzidenz", "Topic_article"] = "infektion"
meine_daten.loc[meine_daten["Topic_article"] == "kontaktbeschränkung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schnelltests", "Topic_article"] = "tests"
meine_daten.loc[meine_daten["Topic_article"] == "verschärfungen", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intesivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "maskenplficht", "Topic_article"] = "maskenpflicht"
meine_daten.loc[meine_daten["Topic_article"] == "fußball", "Topic_article"] = "sport"
meine_daten = meine_daten[meine_daten.Topic_article != "XXX"]
meine_daten.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", header=None, index=None)
meine_daten["Topic_article"].value_counts()

In [ ]:
import pandas as pd
volle_daten= pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
volle_daten.topic_comment.value_counts()